In [101]:
import requests
from shapely.geometry import shape
import geopandas as gpd
import pandas as pd
from shapely.geometry import box

# What is the identifier and spatial geometry for this thing? Does it exist?

In [82]:
# Find all rivers whose name includes “animas”​
query = """
PREFIX schema: <https://schema.org/>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT 
?subject ?name
WHERE {
  # location info
  ?subject rdf:type hyf:HY_FlowPath;
           schema:name ?name.
   
  FILTER (REGEX(?name, "animas", "i"))
  
}
"""
resp = requests.post(
    'https://graph.geoconnex.us/repositories/iow',
    headers={'Accept': 'application/sparql-results+json', 'Content-Type': 'application/sparql-query'},
    data=query
).json()

for result in resp["results"]["bindings"]:
    print(result["subject"]["value"], result["name"]["value"])

https://geoconnex.us/ref/mainstems/1888173 Animas Creek
https://geoconnex.us/ref/mainstems/35394 Animas River
https://geoconnex.us/ref/mainstems/1609859 Las Animas Creek


In [31]:
# Find all Public Water Systems within a bounding box (i.e. in this case a rough bounding box of Arizona)
# Step 1: Load the data
df = gpd.read_file("https://reference.geoconnex.us/collections/pws/items")
df.set_geometry(col="geometry")

# Step 2: Define the bounding box for Arizona
# Approximate coordinates for Arizona
minx, miny, maxx, maxy = -114.818269, 31.332177, -102.041524, 37.00426 
bbox = box(minx, miny, maxx, maxy)

filtered_df = df[df.geometry.within(bbox)]
filtered_df.head()

,id,city_served,tier,source_url,sdwis_link,state_code,county_served,centroid_lat,data_source,place_uri,...,pred_50,boundary_type,pws_name,primacy_type,pred_95,service_area_type,primacy_agency_code,primary_source_code,primary_water_source,geometry
117,063500110,None,2,https://www.hydroshare.org/resource/b11b8982ee...,https://enviro.epa.gov/enviro/sdw_report_v3.fi...,06,None,35.606602,SimpleLab,https://geoconnex.us/ref/places/,...,NaN,Unknown Service Area - Using relevant U.S. Cen...,JEMEZ PUEBLO,Tribal,NaN,Residential Area,06,GW,Groundwater,"MULTIPOLYGON (((-106.7554 35.58712, -106.75059..."
118,063500113,None,2,https://www.hydroshare.org/resource/b11b8982ee...,https://enviro.epa.gov/enviro/sdw_report_v3.fi...,06,None,36.200146,SimpleLab,https://geoconnex.us/ref/places/,...,NaN,Unknown Service Area - Using relevant U.S. Cen...,PICURIS PUEBLO,Tribal,NaN,Residential Area,06,GW,Groundwater,"MULTIPOLYGON (((-105.68716 36.19458, -105.6922..."
120,063500116,None,2,https://www.hydroshare.org/resource/b11b8982ee...,https://enviro.epa.gov/enviro/sdw_report_v3.fi...,06,None,35.888315,SimpleLab,https://geoconnex.us/ref/places/,...,NaN,Unknown Service Area - Using relevant U.S. Cen...,SAN ILDEFONSO PUEBLO,Tribal,NaN,Residential Area,06,GW,Groundwater,"MULTIPOLYGON (((-106.09846 35.90195, -106.1027..."
121,063500119,None,3,https://www.hydroshare.org/resource/b11b8982ee...,https://enviro.epa.gov/enviro/sdw_report_v3.fi...,06,None,35.729218,SimpleLab,https://geoconnex.us/ref/places/,...,725.839092,Unknown Service Area - Using EPA ECHO facility...,NAMBE PUEBLO,Tribal,822.433593,Residential Area,06,GW,Groundwater,"MULTIPOLYGON (((-105.98248 35.72826, -105.9825..."
122,063500120,None,2,https://www.hydroshare.org/resource/b11b8982ee...,https://enviro.epa.gov/enviro/sdw_report_v3.fi...,06,None,35.522443,SimpleLab,https://geoconnex.us/ref/places/,...,NaN,Unknown Service Area - Using relevant U.S. Cen...,SANTO DOMINGO PUEBLO,Tribal,NaN,Residential Area,06,GW,Groundwater,"MULTIPOLYGON (((-106.38049 35.5162, -106.37756..."


In [37]:
# Find which HUC10s my field site is in
minx, miny, maxx, maxy = -104.818269, 35.932177, -104.041524, 36.00426
bbox = f"{minx},{miny},{maxx},{maxy}"
resp = requests.get(f"https://reference.geoconnex.us/collections/hu10/items?bbox={bbox}").json()

for feature in resp["features"]:
    uri = (feature["properties"]["uri"])
    # We can use the stable URI to make another request and get the name from the URI
    name =  requests.get(uri).json()["properties"]["name"]
    print(uri, name)

https://geoconnex.us/ref/hu10/1108000304 Carrizo Creek
https://geoconnex.us/ref/hu10/1108000305 Arroyo Pierda Lumbre-Canadian River
https://geoconnex.us/ref/hu10/1108000405 Middle Mora River
https://geoconnex.us/ref/hu10/1108000601 Headwaters La Cinta Creek
https://geoconnex.us/ref/hu10/1108000702 Alamocita Creek-Ute Creek
https://geoconnex.us/ref/hu10/1108000704 Carrizo Creek
https://geoconnex.us/ref/hu10/1108000705 Tequesquite Creek


# What features are hydrologically related?

In [80]:
# Step 1: Get the geometry from the API response
response = requests.get("https://reference.geoconnex.us/collections/mainstems/items/35394")
geometry = response.json()["geometry"]

# Convert GeoJSON geometry to a Shapely geometry object
shapely_geom = shape(geometry)

# Convert Shapely geometry to WKT
wkt_geom = shapely_geom.wkt

# Step 2: Use WKT in your request
# Construct the request payload
payload = {
    'filter': 'INTERSECTS(geom, POLYGON((-109.448547 36.611118, -109.448547 37.322120, -107.668762 37.322120, -107.668762 36.611118, -109.448547 36.611118)))&limit=1000',
    'limit': 10
}

# Make the POST request
url = "https://reference.geoconnex.us/collections/mainstems/items"
response = requests.post(url, json=payload).json()
response

{'code': 'InvalidParameterValue',
 'type': 'InvalidParameterValue',
 'description': 'Invalid filter language'}

In [91]:
# Find all gages on the “animas river” ​
query = """
PREFIX schema: <https://schema.org/>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?gages WHERE {
  ?gages (hyf:referencedPosition/hyf:HY_IndirectPosition/hyf:linearElement) ?mainstem.
  ?mainstem schema:name ?name.
  FILTER(REGEX(?name, "animas river", "i"))
  ?gages rdf:type hyf:HY_HydrometricFeature.
}
"""
resp = requests.post(
    'https://graph.geoconnex.us/repositories/iow',
    headers={'Accept': 'application/sparql-results+json', 'Content-Type': 'application/sparql-query'},
    data=query
).json()
for result in resp["results"]["bindings"]:
    print(result["gages"]["value"])

https://geoconnex.us/ref/gages/1019578
https://geoconnex.us/ref/gages/1021925
https://geoconnex.us/ref/gages/1147412
https://geoconnex.us/ref/gages/1047611
https://geoconnex.us/ref/gages/1096691
https://geoconnex.us/ref/gages/1069951
https://geoconnex.us/ref/gages/1096785
https://geoconnex.us/ref/gages/1096705
https://geoconnex.us/ref/gages/1061961
https://geoconnex.us/ref/gages/1096753
https://geoconnex.us/ref/gages/1096554
https://geoconnex.us/ref/gages/1070025
https://geoconnex.us/ref/gages/1096334
https://geoconnex.us/ref/gages/1096387
https://geoconnex.us/ref/gages/1013941
https://geoconnex.us/ref/gages/1096461
https://geoconnex.us/ref/gages/1018284
https://geoconnex.us/ref/gages/1096427
https://geoconnex.us/ref/gages/1096371
https://geoconnex.us/ref/gages/1148474
https://geoconnex.us/ref/gages/1020165
https://geoconnex.us/ref/gages/1096789
https://geoconnex.us/ref/gages/1019980
https://geoconnex.us/ref/gages/1071594
https://geoconnex.us/ref/gages/1071212
https://sta.geoconnex.dev

# What dataset is available about a feature?

In [117]:
# SPARQL query to get discharge and temperature datasets
query = """
PREFIX schema: <https://schema.org/>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX wiki: <https://www.wikidata.org/wiki/>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT 
?mainstem 
?monitoringLocation 
?siteName 
?datasetDescription 
?type
?url
?variableMeasured
?variableUnit
?measurementTechnique
?temporalCoverage
?distributionName
?distributionURL
?distributionFormat
?wkt
WHERE {
    VALUES ?mainstem { <https://geoconnex.us/ref/mainstems/29559> }
    
    # location info
    ?monitoringLocation hyf:HydroLocationType ?type;
    		        hyf:referencedPosition/hyf:HY_IndirectPosition/hyf:linearElement ?mainstem;
                        schema:subjectOf ?dataset;
                        gsp:hasGeometry/gsp:asWKT ?wkt .

    # dataset info
    ?dataset  schema:variableMeasured ?var;
              schema:url ?url;
              schema:distribution ?distribution;
              schema:description ?datasetDescription ;
              schema:temporalCoverage ?temporalCoverage ;
              schema:name ?siteName .
    
    # variable metadata
    ?var schema:name ?variableMeasured;
         schema:unitText ?variableUnit;
         schema:measurementTechnique ?measurementTechnique .

    FILTER (REGEX(?variableMeasured, "discharge|temperature", "i"))
    
    # Dataset distribution / download info
    ?distribution schema:name ?distributionName;
                  schema:contentUrl ?distributionURL;
                  schema:encodingFormat ?distributionFormat.
}
LIMIT 100
"""

# Perform the POST request
response = requests.post(
    'https://graph.geoconnex.us/repositories/iow?f=json',
    headers={'Accept': 'application/sparql-results+json', 'Content-Type': 'application/sparql-query'},
    data=query
).json()

print(response)

# Extract the results from the JSON response
results = response['results']['bindings']

# Convert the results to a DataFrame
df = pd.json_normalize(results)

df

{'head': {'vars': ['mainstem', 'monitoringLocation', 'siteName', 'datasetDescription', 'type', 'url', 'variableMeasured', 'variableUnit', 'measurementTechnique', 'temporalCoverage', 'distributionName', 'distributionURL', 'distributionFormat', 'wkt']}, 'results': {'bindings': [{'mainstem': {'type': 'uri', 'value': 'https://geoconnex.us/ref/mainstems/29559'}, 'monitoringLocation': {'type': 'uri', 'value': "https://sta.geoconnex.dev/collections/USGS/Things/items/'USGS-09423000'"}, 'siteName': {'type': 'literal', 'value': 'USGS-09423000'}, 'datasetDescription': {'type': 'literal', 'value': 'Discharge / USGS-09423000-fc344c97ec714e4d91c7828756c66498'}, 'type': {'type': 'literal', 'value': 'Stream'}, 'url': {'type': 'literal', 'value': 'https://waterdata.usgs.gov/monitoring-location/09423000/#parameterCode=00060'}, 'variableMeasured': {'type': 'literal', 'value': 'Discharge / USGS-09423000-fc344c97ec714e4d91c7828756c66498'}, 'variableUnit': {'type': 'literal', 'value': 'ft^3/s'}, 'measuremen

,mainstem.type,mainstem.value,monitoringLocation.type,monitoringLocation.value,siteName.type,siteName.value,datasetDescription.type,datasetDescription.value,type.type,type.value,...,temporalCoverage.value,distributionName.type,distributionName.value,distributionURL.type,distributionURL.value,distributionFormat.type,distributionFormat.value,wkt.datatype,wkt.type,wkt.value
0,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09423000,literal,Discharge / USGS-09423000-fc344c97ec714e4d91c7...,literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.usgs.gov/nwis/iv/?sites=...,literal,text/tab-separated-values,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-114.572187618808 35.1916655605549)
1,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09423000,literal,Discharge / USGS-09423000-fc344c97ec714e4d91c7...,literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-114.572187618808 35.1916655605549)
2,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09185600,literal,Discharge / USGS-09185600-1aed7c25e581411b839a...,literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T17:45:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.usgs.gov/nwis/iv/?sites=...,literal,text/tab-separated-values,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-109.658333333333 38.5047222222222)
3,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09185600,literal,Discharge / USGS-09185600-1aed7c25e581411b839a...,literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T17:45:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-109.658333333333 38.5047222222222)
4,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09060799,literal,"Temperature, water / USGS-09060799-29028343076...",literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-106.831694444444 39.8911)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09019500,literal,"Discharge, cubic feet per second / USGS-090195...",literal,Stream,...,2024-08-30T09:10:00Z/2024-09-09T18:40:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-105.900569365681 40.1208191453499)
66,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09019500,literal,"Temperature, water, degrees Celsius / USGS-090...",literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-105.900569365681 40.1208191453499)
67,uri,https://geoconnex.us/ref/mainstems/29559,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09019500,literal,"Temperature, water, degrees Celsius / USGS-090...",literal,Stream,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.us